# ACSE-4 Project 2: A Smoothed Particle Hydrodynamics Solver

# 1. Description

### a. Algorithm

Our software uses this general class structure. The SPH implementation is in `SPH_stub.py`, this file uses `particle.py` to generate the particle objects. The `simulate` function is the main function that runs in `SPH_stub.py`. It updates the particles in the `particle_list` for each time-step and writes the updated particles object in a pickle file. 

The pickle file can then be used as an input to the `animation.py` script and a mp4 video showing the particles moving in time is generated. 

Our simulation can either be run using the command line interface (with arguments; more details are given in the README file) or for the more savvy python users, we provide scripts that can be edited with the parameters i.e. 

On the command-line, run:

`python run_simulation.py` 

This calls the `simulate` function and creates a pickle file called `datafile.pkl`. 

Then to create the video using this file generated, simpy run:

`python animation.py`


<img src="figures/class_chart.png" style="width: 520px">
<br><br>
The solution algorithm is written as the method *simulate* in *SPH_main*, uses many functions and returns the calculated output as a datafile, according to the diagram below.
<img src="figures/overview_of_simulate_function.png" style="width: 400px">


Our `simulate` function calls the different time-step methods (depending on which is specified). The schemes then call the smoothing function if the time is at the 10th time step. We then call the `neighbour_iterate` function and update the acceleration and the derivative of the density using the `navier_cont` function. 

You will find below the corresponding pseudocode of the method:
<br>
`
Method simulate(self)
    t = t0
    time_array = [t]
    allocate all particles to grid
    cnt = 0
    creating a file
    If a file of the same name already exists:
        delete file
    Generating a progressbar
    i = 0
    count = 0
    max_speed = 0.0   
    While t < t_max:
        cnt = cnt + 1
        smooth = False
        If cnt % 10 == 0:
             Smoothing density must be done
        Perform scheme (forward Euler or Predictor-corrector) and smooth density
        Calculate max speed for printing results
        t = t + dt
        Save file every n dt
        Updating progressbar
    Find out how many particles have leaked from last particle_list
`

### b. How we deal with the output of our simulation

Given that the data volume is pretty huge, in order to save and load it of high efficiency, we import pickle module to save and load files instead of traditional csv. Pickle has the advantage of convenience -- it can serialize arbitrary object with no extra work, and works on a pretty broad range of Python types. It is also smart in that it will only write out any single object once, making it effective to store complex structures because it will write the pickled object in a more efficient binary format, instead of the human-readable format.

The user will get the .pkl format files in the directory. The file saves the output data used for subsequent post-processing and for animation according to the user set simulation time output interval n, which is the parameter defined in the function of simulation. When the simulation function generates the new data, it will be dumped into the file immediately. So even if the function breaks down for some reasons in the midway, we can still obtain the data, which to some extent, improves the sustainability of our software.

### c. Visualisation of our simulation

**Plot the initial state**

Now using `matplotlib.pyplot.plot` to plot all particle position.

The logic goes as: if it is a boundary particle, it becomes black;

             if it is not a boundary particle, it becomes blue.

This time the visualization runs just after data is generate, without read and load file.

**Using plot to generate animation**

Write an animation function to update every state. Update the plot every time when there is a new data.

Now the animation is still doing without saving and loading, just for numerical group members to get a sense of how the particle goes.

The color can't relate to the value such as pressure, velocity now.

The interval should be set as $1000*DataInterval$

**Using ffmpeg to generate the video for animation**

To get a video.mp4 kinds of thing, the animation should be output and transfered to that.

To use ffmpeg tools, first the path of computer should be edited, add the path/bin/ffmpeg.exe to the path then it could work.

The FPS of output is related to the time interval the data is output. For example, if the time interval is $dt$, the FPS should be set as $$1/(dt)$$

**Adding a progressbar**

We really want to know the progress of simulation, so I add a progressbar to show it.

The progressbar is settle in the main class.simuate. Every time one step simulation is done, the progresssbar update the percentage it finished, the time has run and the expected time to finish.

This one make our life easier and always get an idea how fast our simulation is running.

**Merge the file operation**

Add save and load file to fetch the data. Thanks to pickle, the structure of data remain same.

In animation, the data is store in x_data, pressure... All kinds of list for making animation.

The process to make it to animation and output to mp4 is same as before.

**Using scatter plot for color operation**

Because using plt.plot can't change the color of each point every time step freely, I rewrite the animation part by using scatter, the basic logic is the same.

As for color, I define a function to determain them into RGB array. As I want the color to go red when the data value is relatively high and color goes blue when data value is relatively low, I fix the value for Green, set the value for Red goes larger when the value of Blue goes smaller. The relative value of every data is added with abs() to prevent something like negative pressure.

## Improve the stability and add more operation

Create a file to store the number that pickle doing dump and read it when doing loading to improve stability.

Coperate with the numerical part and tried a more rubust way to store the data.

Add some code to make the animation for not only pressure - color, but also velocity and density.

# 2.  Optimisation

### a. Optimising the computational time: Stencil

To make the computational time quicker, we implemented a **stencil** to half the time taken to get the neighbours of each of the particles. 

The diagram below shows us a picture of how the stencil looked and which buckets we will be calculating the neighbours from. 

<img src="figures/stencil.png" style="width: 300px">

The original neighbour searching algorithm looped through every particle and every particle neighbour to find the navier stokes force contribution for each particle from each pair. This algorithm is very inefficient and has a cost complexity of $O(N^2)$. In order to implement a more efficient algorithm we take the knowledge that the acceleration force contribution on particle i from neighbour j, is exactly opposite in direction but same in magnitude onto neighbour j. The contribution to the density term for each particle is the same on i and j.

Using this knowledge we can implement an algorithm where we only ever have to calculate the relationship between two nearby particles once. To do this we only search for neighbouring particles in a particular stencil (see image below), and for every iteration add the force contribution to both the particles and the neighbour. After the whole loop every particle will have aggregated the force contributions from every near neighbour.

One important thing to note:

The forces are only added to the neighbouring particle if the neighbouring particle does not share the same bucket. This is to prevent double duplication of forces in the same bucket. A way we could have further optimised this is to add forces to neighbouring particles in the same bucket but only if the neighbouring particle had not already been read in the for loop.

Overall this stencil searching algorithm roughly halved our computational time for each simulation.

** Video of FE with stencil and without stencil **

Therefore, we can see that our stencil nearly halves the computational time, which was the goal. This is extremely important especially when running on a finer mesh grid (e.g. $\Delta x = 0.2$), since python takes a *very* long time, finding a way to optimise is necessary.

### b. Improving accuracy

The forward-euler method is only first-order method. However, it is necessary for us to have a more accurate method in time as so we implemented the predictor-corrector method. This method is second-order accurate and uses 2 steps to get a full step and is part of the Runga-Kutta family.

The pseudo-code: 
<br>
`
Method predictor_corrector(self, particles, smooth = False)
    For n in range(2):
        If we have to smooth density:
            Smoothing density
        For each particle:
            Applying Navier-Stokes equations and continuity equation
        Clearing grid
    If n == 0:
        For each particle:
            if not a boundary particle:
                store previous values from the particle
                perform predictor-corrector update on position and velocity for half step
                reassign each particle to a bucket
            perform predictor-corrector update on density for half step
            Fill grid and update pressure
            Set acceleration to g initially for next loop
            Set derivative of density to 0 initially for next loop     
    If n == 1:
        For each particle:
            if not a boundary particle:
                perform predictor-corrector update on position and velocity for full step
                reassign each particle to a bucket
            perform predictor-corrector update on density for full step
            Fill grid and update pressure
            Set acceleration to g initially for next loop
            Set derivative of density to 0 initially for next loop
`


We can look at the predictor-corrector method as a 2-step forward euler time step. So, in our code, we used our forward-euler function and ran it twice in a for-loop. We have defined different equations (from half and full step of P-C method) and these equations change according to n in the for-loop.   Also, we added extra attributes to each particle in order to store the velocity and position at the previous half-step. 

#### Comparison between FE and PC schemes

** Parameters used in the videos**

`x_max = (10, 7)
dx = 0.5
t_max = 2 
`

##### Simulation using FE scheme

<video controls src="figures/FE.mp4" />

##### Simulation using PC scheme

<video controls src="figures/Predictor-corrector.mp4" />

These videos show us the difference in behaviour of the particles using the different schemes. The Predictor-Corrector method seems to be more realistic, but runs slower. This is expected as P-C takes 2 half-steps while FE only takes one, so although the accuracy has increased, we lose some computational time. 

The parameters we use are the following:

`
max_x = (10, 7), dx = 0.5, h_fac = 1.3, t0 = 0.0, t_max = 1.5
`
All other parameters are default values.

To generate this simulation, we use FE scheme.
<img src="figures/crest_and_height_time.png" style="width: 700px">

We can see on these figures the location of the crest and the wave height which change with time.
<br>
We calculated the average wave velocity value using this line:
**velocities = np.mean(np.array([np.linalg.norm(p.v) for p in max_particles]))**.
<br>
The average wave velocity calculated from the location of the crest is $2.24 m/s$.
In comparison, the expected shallow water wave speed is of $ \sqrt{gh_0} = 4.90 m/s$.

We notice that the calculated average wave velocity from the simulation is less than the expected wave speed.
For this, there could be some explanations. Indeed, when the simulated wave hits a wall, its velocity decreases, in contrast to the idealised wave which travels freely. Also, the simulated wave's waveshape is obviously not ideal.


# 3. Issues faced

We faced an issue regarding the particles leaking out of the system, as seen in the following video.
<video controls src="figures/leak.mp4" />


We enforced a **repulsive force** on the wall to stop this from happening. The pseudo-code we based our implementation on is written below:

In the `navier_cont` function:
<br>
`
For all particles near wall:
    For all walls:
        normal to wall (vector)
        Calculate distance to wall
        q = distance / (0.75*dx)
        If  0 < q < 1:
            if q < 0.04:
                q = 0.04
            add a value da to the current acceleration
`


We make the system quasi-incompressible by introducing a $P_{ref} =f(\frac{\rho}{\rho_{0}} = 1.02)$, where $f$ is a function. Also, note that we check that the (orthogonal) distance of a fluid particle to the wall is less than $0.9 \Delta x$. We do not want the value to be greater or equal to 1 so that the repulsive force from the wall particles would not affect particles that are not close to them (and not their neighbours). Secondly, we did not choose a value lower to ensure that the particle would be affected by the boundary and not slip through a wall unnoticed. 

However, even with the repulsive force added to the wall of the boundary, this did not prevent some particles from leaking. So we needed a way to handle leaks so that our code does not break every time we have a leak. We decided to let the particles leave the boundary without having errors. As we will see later in the main demonstration, the repulsive force implementation can cause some interesting errors regarding the leakage of the particles. 

# 4. Demonstration of the software

** Plotting the pressure of the particles each time-step **
<video controls src="figures/Pressure_video.mp4" />

** Plotting the density of the particles each time-step **
<video controls src="figures/Rho_video.mp4" />

** Plotting the velocity of the particles each time-step **
<video controls src="figures/Velocity_video.mp4" />

### The issue with very fine initial grid space

When we test with $\Delta{x} = 0.2$, we get the following video: 
<video controls src="figures/Velocity_video.mp4" />

**Instability**

We have found that our code has a large instality when run on a fine intial dx space. The instability is initiated by particles that clump on the left wall. These particles clearly have an atractive force to the wall and start ocillating towards the wall. The particles also develop an anomously low density due to being isolated from the main body of water.

The boundary forcing pushes the particles out from the wall, and the anonamously low density of the particles causes a large attractive force from the particles to the wall. These particales eventually oscillate so close to the wall that the boundary forcing term is so large is reppells the particles with enormous force into the rest of the fluid body and completely disrupts the simulation.

# 5. Checking the software output using the expected shallow water wave speed

# 6. Convergence analysis

To do the convergence study, we change the resolution $\Delta{x}$ and we check the time when the wave traverse the domain and hits the wall for the first time.
Since we were running out of time, we decided to calculate the convergence of $\Delta{x}$ and time using a very approximate method. We would generate the video of the simulation with stop time approximately after the wave hits the boundary.
We would then "eyeball" the time where the wave would hit the boundary. 

Admittedly, with a larger $\Delta{x}$, estimating the time was easier to do and got progressively harder as the $\Delta{x}$ decreased since there were more particles.

<img src="figures/convergence_analysis.png" style="width: 500px">

According to a technical report by Hu, Pan, Rakhsha and Negrut, SPH method is second-order accurate. Our convergence analysis shows an order of accuracy of 1.45. Such a result could be explained by our very approximative method to get our time values at different resolutions, which prevents us to get the desired order of 2.

# 7. Artificial Pressure

A common issue with SPH codes is the “clumping” of fluid structures into local groups. In order to combat this problem there has been proposal of a addition of an artificial pressure term in the literature:

https://ac.els-cdn.com/S0021999100964398/1-s2.0-S0021999100964398-main.pdf?_tid=a6294837-e03d-4468-bc06-1acdf05cd854&acdnat=1544786533_9804250bcc3ad6c4fa850effddf67d25
(Monaghan, J. (2000). SPH without a Tensile Instability. Journal of Computational Physics, 159(2), pp.290-311.).

The equations proposed here add a forcing term when particles get too close to eachother. The term utilises the same smoothing kernel as used in our main code so is very easy to integrate into our function at each particle-neighbour interaction.

The additional term  $$ Rf_{ab}^{n} \ \ in \ equation \ 2.4 \ from \ paper $$

This term adds a repulsive force to each particle when they get too close.
We use the n parameter as 4. We tune the scale of the force to get the desired result.

A problem we encountered was finding the right scale to the force. Too small and the particles still show clumping. Too large and the forces cause instabillities in the simulation which end up forcing all particles out of the domain. In the end it seemed there was no perfect answer and we found that we had to allow a slight level of clumping to maintain stable code. 